In [ ]:
import geemap
import ee
import os
import time
import pandas as pd

In [ ]:
email = "mt-hvi-zonalstats-service-acco@mt-hvi-zonalstats.iam.gserviceaccount.com"
key_file = "/Users/natebender/Desktop/repo/mt-hvi-zonalstats-6da42ca28c80.json"

# Authenticate and initialize
credentials = ee.ServiceAccountCredentials(email=email, key_file=key_file)
ee.Initialize(credentials)

In [ ]:
# Function to apply scale factors to the Landsat data
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_band, None, True)

# Function to mask clouds based on the pixel_qa band of Landsat SR data.
# def cloud_mask(image):
#     qa = image.select('QA_PIXEL')
#     cloud = qa.bitwiseAnd(1 << 3).neq(0)
#     return image.updateMask(cloud.Not())

# def compute_cloud_cover(image):
#     cloud = image.select('QA_PIXEL').bitwiseAnd(1 << 3).neq(0)
#     cloud_area = cloud.reduceRegion(
#         reducer=ee.Reducer.sum(),
#         geometry=aoi,
#         scale=30,
#         maxPixels=1e13
#     ).get('QA_PIXEL')
    
#     total_area = ee.Number(image.select('QA_PIXEL').unmask().reduceRegion(
#         reducer=ee.Reducer.count(),
#         geometry=aoi,
#         scale=30,
#         maxPixels=1e13
#     ).get('QA_PIXEL'))
    
#     cloud_cover_percentage = ee.Number(cloud_area).divide(total_area).multiply(100)
#     return image.set('CLOUD_COVER_PERCENTAGE', cloud_cover_percentage)

# def process_with_delay(collection):
#     results = []
#     for image in collection.toList(collection.size()).getInfo():
#         img = ee.Image(image['id'])
#         img = cloud_mask(img)
#         img = compute_cloud_cover(img)
#         results.append(img)
#         #time.sleep(1)  # Add a 1 second delay between processing each image to avoid rate limits.
#     return ee.ImageCollection(results)

def print_collection_summary(collection):
    images = collection.toList(collection.size()).getInfo()
    for image in images:
        id = image['id']
        date = image['properties']['DATE_ACQUIRED']
        print(f"Image ID: {id}, Date: {date}")

def kelvin_to_fahrenheit(image):
    # Convert the ST_B10 band from Kelvin to Fahrenheit.
    fahrenheit = image.select('ST_B10').subtract(273.15).multiply(9/5).add(32)
    return image.addBands(fahrenheit.rename('ST_Fahrenheit'))


In [ ]:
# Define the area of interest (aoi) as the boundary of Montana.
aoi = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "MT"))
tracts = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq("STATEFP", "30"))

# Get the Landsat 8 image collection.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2018-06-01', '2022-08-31') \
    .filter(ee.Filter.calendarRange(6, 8, 'month')) \
    .map(apply_scale_factors) 

In [ ]:
start_time = time.time()
num_images = collection.size().getInfo()
print(f"Total number of images: {num_images}")
#print_collection_summary(collection) # print this to see all the image dates
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
start_time = time.time()
# Process the collection with a delay to handle rate limiting.
filtered_collection = collection.filter(ee.Filter.lte('CLOUD_COVER', 10)) #lte stands for less than or equal to
num_images = filtered_collection.size().getInfo()
print(f"Total number of images: {num_images}")
#print_collection_summary(filtered_collection) # print this to see all the image dates

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
bad_composite = collection.mean().clip(aoi)
good_composite = filtered_collection.mean().clip(aoi)

In [ ]:
# check impact of filtering by cloud cover
m = geemap.Map()
m.set_center(-110, 47, 6)

# Convert the composites to Fahrenheit.
bad_composite_f = kelvin_to_fahrenheit(bad_composite)
good_composite_f = kelvin_to_fahrenheit(good_composite)

vis_params = {'min': 30, 'max': 120,     
              'palette': ['000004', '1b0c41', '4a0c6b', '781c6d', 'a52c60', 
                          'cf4446', 'ed6925', 'fb9b06', 'f7d13d', 'fcffa4']
             }

# Define the legend dictionary
legend_dict = {
    '30-40°F': '#000004',
    '40-50°F': '#1b0c41',
    '50-60°F': '#4a0c6b',
    '60-70°F': '#781c6d',
    '70-80°F': '#a52c60',
    '80-90°F': '#cf4446',
    '90-100°F': '#ed6925',
    '100-110°F': '#fb9b06',
    '110-120°F': '#f7d13d',
    '120+°F': '#fcffa4'
}

# Add the Fahrenheit layer for bad_composite.
m.add_layer(
    bad_composite_f,
    {'bands': ['ST_Fahrenheit'], **vis_params},
    'Bad composite (F)',
)

# Add the Fahrenheit layer for good_composite.
m.add_layer(
    good_composite_f,
    {'bands': ['ST_Fahrenheit'], **vis_params},
    'Avg Summer Temp, 2018-2022',
)

# Add the Montana boundary to the map with styling.
m.add_layer(
    aoi.style(color='black', fillColor='00000000'),  # '00000000' represents no fill color
    {},
    'Montana',
)

# add Census Tracts
m.add_layer(
    tracts.style(color='black', fillColor='00000000'),  # '00000000' represents no fill color
    {},
    'Tracts',
)

m.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
m.add_legend(title="Temperature (°F)", legend_dict=legend_dict)

m

In [ ]:
start_time = time.time()
mean_dict = good_composite_f.select('ST_Fahrenheit').reduceRegions(
    collection=tracts,
    reducer=ee.Reducer.mean(),
    scale=30
)
# Print the result (limited to first few tracts for brevity).
print(mean_dict.first().getInfo())
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
results = mean_dict.getInfo()['features']
data = []
for feature in results:
    props = feature['properties']
    data.append({
        'GEOID': props['GEOID'],
        'NAME': props['NAME'],
        'mean_temperature_F': props['mean']
    })

# Convert the list of dictionaries to a pandas DataFrame.
df = pd.DataFrame(data)


In [ ]:
df.head()

In [ ]:
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, 'tract_avg_summertemp_2018-22.csv')
df.to_csv(csv_path, index=False)